In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch_geometric.nn as gnn
import pickle
from tqdm import tqdm
import numpy as np
from torch.functional import F
import matplotlib.pyplot as plt


with open("data/concat_graph/train_graphs_data.pkl", "rb") as file:
    graphs = pickle.load(file)
    file.close()

In [6]:
len(graphs), graphs[0]

(84, Data(x=[7980, 33], edge_index=[2, 85761], y=[7980, 2]))

In [7]:
graphs[0].y[0]

tensor([-0.2741, -0.6940])

In [8]:
# # 划分训练集和验证集，使用sklearn的train_test_split函数
# from sklearn.model_selection import train_test_split

# train_graphs, val_graphs = train_test_split(graphs, test_size=0.1, random_state=42)
val_len = graphs[0].x.shape[0] // 1140
train_graphs = graphs[:-val_len]
val_graphs = graphs[-val_len:]
len(train_graphs), len(val_graphs)

(77, 7)

In [9]:
class SAGEModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SAGEModel, self).__init__()

        self.graph_features = nn.ModuleList(
            [
                gnn.SAGEConv(input_size, 128),
                nn.LayerNorm(128),
                nn.ReLU(),
                gnn.SAGEConv(128, 128),
                nn.LayerNorm(128),
                nn.ReLU(),
                gnn.SAGEConv(128, 128),
                nn.LayerNorm(128),
                nn.ReLU(),
            ]
        )

        # 考虑更多的trick，如layernorm等等
        self.regression = nn.Sequential(
            nn.Linear(128, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.LayerNorm(32),
            nn.ReLU(),
            nn.Dropout(0.02),
            nn.Linear(32, output_size),
        )

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        output = x

        for layer in self.graph_features:
            if isinstance(layer, gnn.SAGEConv):
                output = layer(output, edge_index)
            else:
                output = layer(output)

        output = self.regression(output)

        return output

In [10]:
# 看看模型的输入输出
model = SAGEModel(input_size=33, output_size=2)
output = model(graphs[0])
output.shape

torch.Size([7980, 2])

In [11]:
input_size = graphs[0].x.shape[1]
# 这里的output_size就是最终的输出维度，不需要再乘以num_heads
output_size = graphs[0].y.shape[1]
# 个人认为，注意力头的个数应该和输出维度保持一致，因为每个输出可能需要关注不同的邻居
num_heads = graphs[0].y.shape[1]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# batch_size = 256

# 开始训练

In [12]:
def train(
    model, criterion, optimizer, scheduler, train_graphs, val_graphs=None, num_epochs=50
):
    # 训练过程记录
    train_loss_list = []
    val_loss_list = []

    with tqdm(total=num_epochs, desc="Training Progress", unit="epoch") as pbar_epochs:
        for epoch in range(num_epochs):
            # 训练
            model.train()
            train_loss = 0.0

            # 目前每个batch是1个graph
            for i, graph in enumerate(train_graphs):
                graph = graph.to(device)

                output = model(graph)

                optimizer.zero_grad()
                loss = criterion(output, graph.y)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

            train_loss /= len(train_graphs)
            train_loss_list.append(train_loss)

            # 验证
            if val_graphs:
                model.eval()
                with torch.no_grad():
                    val_loss = 0.0
                    for i, graph in enumerate(val_graphs):
                        graph = graph.to(device)
                        output = model(graph)
                        loss = criterion(output, graph.y)
                        val_loss += loss.item()
                    val_loss /= len(val_graphs)
                    val_loss_list.append(val_loss)

            if val_graphs:
                pbar_epochs.set_postfix(
                    {"train MSE Loss": train_loss, "val MSE Loss": val_loss}
                )
            else:
                pbar_epochs.set_postfix({"train MSE Loss": train_loss})

            pbar_epochs.update(1)
            # 学习率更新
            scheduler.step()

    # 可视化训练过程
    plt.figure()
    plt.plot(train_loss_list, label="train loss")
    if val_graphs:
        plt.plot(val_loss_list, label="val loss")
    plt.xlabel("epoch")
    plt.ylabel("MSE loss")
    plt.legend()

    return model

In [13]:
# 定义模型、算法、损失函数
model = SAGEModel(
    input_size=input_size,
    output_size=output_size,
).to(device)
# 考虑是否加入weight_decay
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
# 学习率衰减
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
# criterion = nn.MSELoss(reduction="sum")
criterion = nn.MSELoss().to(device)

# model = train(
#     model=model,
#     criterion=criterion,
#     optimizer=optimizer,
#     scheduler=scheduler,
#     train_graphs=train_graphs,
#     val_graphs=val_graphs,
#     num_epochs=150,
# )

# 所有数据放进去训一训

In [14]:
# 定义模型、算法、损失函数
model = SAGEModel(
    input_size=input_size,
    output_size=output_size,
).to(device)
# 考虑是否加入weight_decay
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
# 学习率衰减
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
# criterion = nn.MSELoss(reduction="sum")
criterion = nn.MSELoss().to(device)

# model = train(
#     model=model,
#     criterion=criterion,
#     optimizer=optimizer,
#     scheduler=scheduler,
#     train_graphs=graphs,
#     num_epochs=150,
# )

In [15]:
# 保存模型
model_path = "saved/models/sage_concat.pth"
torch.save(model, model_path)

# 预测

In [16]:
# 读取模型
model = torch.load(model_path).to(device)

In [17]:
# 读取node_ids
with open("data/concat_graph/node_ids.pkl", "rb") as file:
    node_ids = pickle.load(file)
    file.close()

In [18]:
len(node_ids)

7980

In [19]:
# 读取scaler
import pickle

with open("saved/scaler/label_scaler.pkl", "rb") as file:
    label_scaler = pickle.load(file)
    file.close()

In [20]:
# 读取测试集
with open("data/concat_graph/test_graphs_data.pkl", "rb") as file:
    test_graphs = pickle.load(file)
    file.close()

In [21]:
len(test_graphs), test_graphs[0]

(4, Data(x=[7980, 33], edge_index=[2, 140370]))

预测

In [22]:
model.eval()
output_list = []
with torch.no_grad():
    for i, graph in enumerate(test_graphs):
        graph = graph.to(device)
        output = model(graph)
        print(output.shape)
        # output_list.append(output.detach().cpu().numpy()[-len(node_ids) // 1140 :, :])
        # print(output_list[-1].shape)
        break

/opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [176,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [176,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [176,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [176,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [176,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/cond

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
len(output_list), output_list[0].shape, output_list[0][0]

(4, (1140, 2), array([-0.43294135, -1.0331519 ], dtype=float32))

In [ ]:
# 缩放
output_list = [label_scaler.inverse_transform(output) for output in output_list]

In [ ]:
# 看看是不是缩放完成啦
output_list[0][0]

array([68.76068 , 60.836132], dtype=float32)

### 把输出转成需要的格式

In [ ]:
import csv

# 结果写入csv，分隔符为\t
output_path = "output/sage_out.csv"

date_id = [20230404, 20230405, 20230406, 20230407]
with open(output_path, "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")  # 设置分隔符为制表符
    writer.writerow(["geohash_id", "consumption_level", "activity_level", "date_id"])

    # output_list.shape = [4天, 节点数, 2个输出]
    for nidx in range(len(node_ids)):
        for day in range(len(date_id)):
            # 注意不要写反了
            writer.writerow(
                [
                    node_ids[nidx],
                    output_list[day][nidx][1],
                    output_list[day][nidx][0],
                    date_id[day],
                ]
            )
    f.close()